# Code Testing

This notebook should contain tests showing that for hermitian matrices of sizes up to 30-by-30 with known eigenvalues, the function hermitian eigensystem gives correct eigenvalues and eigenvectors.

First, let's import our packages and functions we will use. 

`proj_2_module.py` contains code to diagonalize a Hermitian matrix. 

`proj_2_EigenTest.py` contains helper functions to test code and verify our calculations.

In [11]:
from proj_2_module import hermitian_eigensystem
from proj_2_EigenTest import npprint, gen_rand_herm, is_hermitian, is_eigenvector, verify_eigenvectors
import numpy as np
import time

It is my opinion that the default __str__ representation for numpy arrays is not that pretty. So throughout this project, I will use my own method, `npprint`, to display arrays. This uses numpy's `printoptions` to display the given array. 

Here is an example. 

In [2]:
_, a = gen_rand_herm(3)

print("This is what the default print looks like:")
print(a)
print("\nThis is what npprint looks like:")
npprint(a)

This is what the default print looks like:
[[-3.06979823+2.77555756e-17j  0.56069676-2.38761630e+00j
  -0.10489019-5.67376079e-01j]
 [ 0.56069676+2.38761630e+00j -3.2794576 -1.11022302e-16j
   1.49902027-8.07285611e-01j]
 [-0.10489019+5.67376079e-01j  1.49902027+8.07285611e-01j
  -3.65074417+0.00000000e+00j]]

This is what npprint looks like:
[[-3.0698+0.j      0.5607-2.3876j -0.1049-0.5674j]
 [ 0.5607+2.3876j -3.2795-0.j      1.499 -0.8073j]
 [-0.1049+0.5674j  1.499 +0.8073j -3.6507+0.j    ]]


As you can see, it helps a lot for floats when a row can wrap to the next line. 

## Matrix Basics

Before we go full gung ho into testing our functions, I want better explain some things that will help when learning this project from scratch. 

A Matrix $A$ is considered Symmetric if $A = A^T$, where $A^T$ is the transpose of $A$.

A Matrix $A$ is considered Hermitian if $A=A^H=(A^*)^T$ where $A^*$ is the complex conjugate of $A$. 

In `numpy`, the transpose of a matrix `a` is `a.T`. The conjugate transpose of a matrix `a` is `a.conj().T`.

A square matrix $A$ is [diagonalizable](https://en.wikipedia.org/wiki/Diagonalizable_matrix) if there exists an invertible matrix $V$ and a diagonal matrix $D$ such that $A = V D V^{-1}$. In this project, we are interested in diagonlizable Hermitian matrices where $A = V D V^{*}$. 

## Generating Random Hermitian Matricies

In order to test our functions, we want to randomly produce Hermitian matricies. The method `gen_rand_herm` does just that. For example, we can produce a random 3x3 Hermitian matrix with known complex eigenvalues.

For nice numbers, I designed `gen_rand_herm` to produce matricies with eigenvalues of form $a + bi$ where $a$ and $b$ are integers. 

In [3]:
e, a = gen_rand_herm(dim = 3)

print("Random eigenvalues:")
npprint(e)

print("\nRandom Hermitian Matrix:")
npprint(a)

Random eigenvalues:
[9 5 1]

Random Hermitian Matrix:
[[ 3.0008+0.j     -0.802 +1.641j   0.6056-1.8231j]
 [-0.802 -1.641j   7.9048-0.j      0.5177+1.4415j]
 [ 0.6056+1.8231j  0.5177-1.4415j  4.0943+0.j    ]]


Let's verify that `gen_rand_herm` is actually Hermitian. 

In [4]:
print(is_hermitian(a))

True


## Diagonalizing Hermitian Matricies

Great, we are able to produce random Hermitian Matricies. One thing to note is that if $A$ is Hermitian, then it must have real eigenvalues. See [Claim 1](https://users.cs.duke.edu/~kamesh/ModernAlg/lec5.pdf). 

Numpy has a built in method, `numpy.linal.eigh`, that will compute the eigenvalues and normalized eigenvector matrix. Our goal is to produce a method that will output similar using the [Jacobi Eigenvalue Algorithm](https://en.wikipedia.org/wiki/Jacobi_eigenvalue_algorithm).

Let's see what numpy calculates for the eigenvalues for a matrix with known eigenvalues. 


The matrix 
$$\begin{bmatrix}
1 & \sqrt{2} & 2\\ 
\sqrt{2} & 3 & \sqrt{2}\\ 
2 & \sqrt{2} & 1
\end{bmatrix}$$ 
has eigenvalues $(-1, 1, 5)$.

In [5]:
test_array = np.array([[1, 2**0.5, 2],[2**0.5, 3, 2**0.5],[2, 2**0.5, 1]])

w, v = np.linalg.eigh(test_array)

print("test_array has eigenvalues: ")
print(w)

test_array has eigenvalues: 
[-1.  1.  5.]


Now let's verify that the random Hermitian matricies we are producing have the eigenvalues we expect them to have.

In [6]:
e, a = gen_rand_herm(dim = 3)

print("Random eigenvalues:")
npprint(np.sort(e*1.0))

w, v = np.linalg.eigh(a)

print("Generated Hermitian has eigenvalues: ")
npprint(w)

Random eigenvalues:
[-9.000 0.000 5.000]
Generated Hermitian has eigenvalues: 
[-9.000 -0.000 5.000]


Indeed it does. We can now confirm that our `gen_rand_herm` method produces random Hermitian matricies with known eigenvalues. We can now use these random Hermitian matricies to test the Jacobi method we wrote. 

## Verify Jacobi Diagonalization Method

I have written a method, `hermitian_eigensystem`, to diagonalize Hermitian matricies. Let's verify that this method produces valid solutions. 

First, generate a random 3x3 Hermitian matrix $A$. Then use `hermitian_eigensystem` and `np.linalg.eigh` to calculate solutions for $A$. We should verify that they produce the same eigenvalues. 

Then, we should verify that the eigenvectors given by `hermitian_eigensystem` are indeed eigenvectors with the corresponding eigenvalue pairs. 

In [8]:
# Generate random Hermitian matrix and eigenvalue set
e, a = gen_rand_herm(3)
print("The expected eigenvalues are:")
npprint(np.sort(e))
print()

my_w, my_v = hermitian_eigensystem(a)
np_w, np_v = np.linalg.eigh(a)

print("Jacobi method eigenvalues:")
npprint(my_w)
print("Numpy method eigenvalues:")
npprint(np_w)
print("Do the eigenvalues match?")
print(np.allclose(my_w, np_w))
print()

print("Do the eigenvectors match the eigenvalues produced by the Jacobi method?")
print(verify_eigenvectors(a, my_w, my_v))


The expected eigenvalues are:
[-6 -6  7]

Jacobi method eigenvalues:
[-6.000 -6.000 7.000]
Numpy method eigenvalues:
[-6.000 -6.000 7.000]
Do the eigenvalues match?
True

Do the eigenvalues match the eigenvalues produced by the Jacobi method?
True


Excellent, everything seems to be working as expected. Now let's test our Jacobi method with Hermitian matricies of different sizes ranging from 2x2 to 30x30. 

In [17]:
for dim in range(2, 31):
    e, a = gen_rand_herm(dim)
    tstart = time.time()
    w, v = hermitian_eigensystem(a)
    tend = time.time()
    eigenvalues_match = np.allclose(w, np.sort(e))
    eigenvectors_match = verify_eigenvectors(a, w, v)
    if eigenvalues_match and eigenvectors_match:
        dtime = tend - tstart
        print(f"Diagonalized {dim}x{dim} Hermitian matrix in {dtime:.2f} seconds.")
    else:
        print(f"Failed to diagonalize {dim}x{dim} Hermitian matrix.")

Diagonalized 2x2 Hermitian matrix in 0.00 seconds.
Diagonalized 3x3 Hermitian matrix in 0.00 seconds.
Diagonalized 4x4 Hermitian matrix in 0.00 seconds.
Diagonalized 5x5 Hermitian matrix in 0.01 seconds.
Diagonalized 6x6 Hermitian matrix in 0.02 seconds.
Diagonalized 7x7 Hermitian matrix in 0.03 seconds.
Diagonalized 8x8 Hermitian matrix in 0.05 seconds.
Diagonalized 9x9 Hermitian matrix in 0.08 seconds.
Diagonalized 10x10 Hermitian matrix in 0.10 seconds.
Failed to diagonalize 11x11 Hermitian matrix.
Diagonalized 12x12 Hermitian matrix in 0.22 seconds.
Failed to diagonalize 13x13 Hermitian matrix.
Failed to diagonalize 14x14 Hermitian matrix.
Diagonalized 15x15 Hermitian matrix in 0.44 seconds.
Failed to diagonalize 16x16 Hermitian matrix.
Diagonalized 17x17 Hermitian matrix in 0.71 seconds.
Diagonalized 18x18 Hermitian matrix in 0.90 seconds.
Failed to diagonalize 19x19 Hermitian matrix.
Failed to diagonalize 20x20 Hermitian matrix.
Failed to diagonalize 21x21 Hermitian matrix.
Faile